# 데이터 로드

로컬에서 저장된 파일 링크를 사용하여 데이터 로드하였습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
pip install pulp

In [ ]:
import pulp

In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, value

In [ ]:
path = pd.read_excel('/content/drive/MyDrive/DART-B/대회용/시나리오테이블(fromtofiltered)-창고비수정.xlsx',sheet_name=None)

# 재고량 증가 반영 전

창고 수 1개 -> 3개 증가에 따른 재고 증가율을 반영하지 않은 경우에 대한 코드입니다.

In [ ]:
# 엑셀 파일에서 데이터 로드
warehouse_data = path['WHCandidatdeCity']  # 창고 후보지 데이터
distance_data = path['FROMTO(Q2)']  # 거리 데이터
city_data = path['City']  # 수요 도시 데이터

# inbound_cost와 warehouse_cost를 딕셔너리로 변환
inbound_cost = {row['City&State'].strip().upper(): row['Inbound'] for _, row in warehouse_data.iterrows()}
warehouse_cost = {row['City&State'].strip().upper(): row['Warehouse'] for _, row in warehouse_data.iterrows()}

# 거리, 수요량, TruckUnitRate를 포함한 delivery_cost 딕셔너리 생성
delivery_cost = {
    (row['From City&State'].strip().upper(), row['To City&State'].strip().upper()): {
        'distance': row['TempD(직각거리)'],
        'demand': row['Demand'],
        'truck_unit_rate': row['TruckUnitRate']
    }
    for _, row in distance_data.iterrows()
}

# 최적화 문제 설정
model = LpProblem("Warehouse_Location_Optimization", LpMinimize)

# 결정 변수: 각 창고를 선택할지 여부 (0 또는 1)
warehouses = warehouse_data['City&State'].str.strip().str.upper().tolist()
x = LpVariable.dicts("SelectWarehouse", warehouses, cat='Binary')  # 창고 선택 변수

# 결정 변수: 각 도착 지역이 어떤 창고에서 물건을 받을지를 나타내는 변수 (0 또는 1)
to_cities = city_data['City&State'].str.strip().str.upper().tolist()
y = {
    (to_city, warehouse): LpVariable(f"Assign_{to_city}_to_{warehouse}", cat='Binary')
    for to_city in to_cities for warehouse in warehouses
}

# 목표 함수 설정: 총 물류비 최소화
model += lpSum([
    y[(to_city, warehouse)] * (
        delivery_cost[(warehouse, to_city)]['distance'] *
        delivery_cost[(warehouse, to_city)]['demand'] *
        delivery_cost[(warehouse, to_city)]['truck_unit_rate']
    )
    for to_city in to_cities
    for warehouse in warehouses
    if (warehouse, to_city) in delivery_cost
]) + lpSum([x[warehouse] * (inbound_cost[warehouse] + warehouse_cost[warehouse]) for warehouse in warehouses])

# 제약 조건 1: 각 도착 지역은 하나의 창고에서만 물건을 받을 수 있음
for to_city in to_cities:
    model += lpSum([y[(to_city, warehouse)] for warehouse in warehouses]) == 1

# 제약 조건 2: 창고가 선택되지 않으면 해당 창고로 할당할 수 없음
for warehouse in warehouses:
    for to_city in to_cities:
        model += y[(to_city, warehouse)] <= x[warehouse]

# 제약 조건 3: 정확히 3개의 창고를 선택
model += lpSum([x[warehouse] for warehouse in warehouses]) == 3

# 최적화 문제 해결
model.solve()

# 선택된 창고와 비용 계산
selected_warehouses = [warehouse for warehouse in warehouses if x[warehouse].value() == 1]

# 결과 계산 및 데이터프레임 생성
results = []
total_inbound_cost = 0
total_warehouse_cost = 0
total_delivery_cost = 0

for warehouse in selected_warehouses:
    # 각 창고에 대한 비용 계산
    inbound = inbound_cost[warehouse]
    warehouse_op = warehouse_cost[warehouse]
    delivery = sum(
        delivery_cost[(warehouse, to_city)]['distance'] *
        delivery_cost[(warehouse, to_city)]['demand'] *
        delivery_cost[(warehouse, to_city)]['truck_unit_rate']
        for to_city in to_cities
        if (warehouse, to_city) in delivery_cost and y[(to_city, warehouse)].value() == 1
    )

    total_cost = inbound + warehouse_op + delivery
    total_inbound_cost += inbound
    total_warehouse_cost += warehouse_op
    total_delivery_cost += delivery

    results.append({
        "후보 창고 도시명": warehouse,
        "수요 박스 개수": sum(delivery_cost[(warehouse, to_city)]['demand'] for to_city in to_cities if (warehouse, to_city) in delivery_cost and y[(to_city, warehouse)].value() == 1),
        "운송비": delivery,
        "수입 물류비": inbound,
        "창고비": warehouse_op,
        "총 물류비": total_cost
    })

# 총 비용 계산
total_costs = {
    "후보 창고 도시명": "총합계",
    "수요 박스 개수": sum(item["수요 박스 개수"] for item in results),
    "운송비": total_delivery_cost,
    "수입 물류비": total_inbound_cost,
    "창고비": total_warehouse_cost,
    "총 물류비": total_inbound_cost + total_warehouse_cost + total_delivery_cost
}
results.append(total_costs)

# 결과를 데이터프레임으로 출력
df_before = pd.DataFrame(results)
pd.options.display.float_format = '{:,.2f}'.format
df_before

,후보 창고 도시명,수요 박스 개수,운송비,수입 물류비,창고비,총 물류비
0,"AUSTIN, TX",2177041,"2,282,487.11","433,738.07","30,456.07","2,746,681.25"
1,"LOS ANGELES, CA",2751355,"2,889,411.13","955,165.00","187,240.11","4,031,816.24"
2,"PITTSBURGH, PA",3871611,"3,974,008.34","165,375.00","9,769.28","4,149,152.62"
3,총합계,8800007,"9,145,906.58","1,554,278.07","227,465.46","10,927,650.10"


#재고량 증가 반영 후

창고 수 1개 -> 3개 증가에 따른 재고 증가율을 반영한 이후의 경우에 대한 코드입니다.

In [ ]:
# 엑셀 파일에서 데이터 로드 (가정: 이미 로드된 warehouse_data 및 distance_data)
warehouse_data = path['WHCandidatdeCity']  # 창고 후보지 데이터
distance_data = path['FROMTO(Q2)']  # 거리 데이터
city_data = path['City']  # 수요 도시 데이터

# inbound_cost와 warehouse_cost를 딕셔너리로 변환
inbound_cost = {row['City&State'].strip().upper(): row['Inbound'] for _, row in warehouse_data.iterrows()}
warehouse_cost = {row['City&State'].strip().upper(): row['Warehouse'] for _, row in warehouse_data.iterrows()}

# 거리, 수요량, TruckUnitRate를 포함한 delivery_cost 딕셔너리 생성
delivery_cost = {
    (row['From City&State'].strip().upper(), row['To City&State'].strip().upper()): {
        'distance': row['TempD(직각거리)'],
        'demand': row['Demand'],
        'truck_unit_rate': row['TruckUnitRate']
    }
    for _, row in distance_data.iterrows()
}

# 최적화 문제 설정
model = LpProblem("Warehouse_Location_Optimization", LpMinimize)

# 결정 변수: 각 창고를 선택할지 여부 (0 또는 1)
warehouses = warehouse_data['City&State'].str.strip().str.upper().tolist()
x = LpVariable.dicts("SelectWarehouse", warehouses, cat='Binary')  # 창고 선택 변수

# 결정 변수: 각 도착 지역이 어떤 창고에서 물건을 받을지를 나타내는 변수 (0 또는 1)
to_cities = city_data['City&State'].str.strip().str.upper().tolist()
y = {
    (to_city, warehouse): LpVariable(f"Assign_{to_city}_to_{warehouse}", cat='Binary')
    for to_city in to_cities for warehouse in warehouses
}

# 목표 함수 설정: 총 물류비 최소화
model += lpSum([
    y[(to_city, warehouse)] * (
        delivery_cost[(warehouse, to_city)]['distance'] *
        delivery_cost[(warehouse, to_city)]['demand'] *
        delivery_cost[(warehouse, to_city)]['truck_unit_rate']
    )
    for to_city in to_cities
    for warehouse in warehouses
    if (warehouse, to_city) in delivery_cost
]) + lpSum([x[warehouse] * (inbound_cost[warehouse] + warehouse_cost[warehouse]) for warehouse in warehouses])

# 제약 조건 1: 각 도착 지역은 하나의 창고에서만 물건을 받을 수 있음
for to_city in to_cities:
    model += lpSum([y[(to_city, warehouse)] for warehouse in warehouses]) == 1

# 제약 조건 2: 창고가 선택되지 않으면 해당 창고로 할당할 수 없음
for warehouse in warehouses:
    for to_city in to_cities:
        model += y[(to_city, warehouse)] <= x[warehouse]

# 제약 조건 3: 정확히 3개의 창고를 선택
model += lpSum([x[warehouse] for warehouse in warehouses]) == 3

# 최적화 문제 해결
model.solve()

# 선택된 창고와 비용 계산
selected_warehouses = [warehouse for warehouse in warehouses if x[warehouse].value() == 1]

# 기본 재고량 설정 (예: 단일 창고일 때의 총 수요량 기반)
single_warehouse_inventory = sum(
    delivery_cost[(warehouse, to_city)]['demand']
    for to_city in to_cities
    for warehouse in selected_warehouses
    if (warehouse, to_city) in delivery_cost
)

# 창고 수 증가에 따른 재고 증가율 계산
inventory_increase_factor = (3 ** 0.5)  # 창고 수 3개일 때 제곱근 법칙 적용

# 결과 계산 및 데이터프레임 생성
results = []
total_inbound_cost = 0
total_warehouse_cost = 0
total_delivery_cost = 0

for warehouse in selected_warehouses:
    # 각 창고에 대한 비용 계산
    inbound = inbound_cost[warehouse]

    # 창고 수 증가에 따른 재고 증가 감안 창고비용 계산
    base_warehouse_cost = warehouse_cost[warehouse]
    adjusted_warehouse_cost = base_warehouse_cost * inventory_increase_factor

    delivery = sum(
        delivery_cost[(warehouse, to_city)]['distance'] *
        delivery_cost[(warehouse, to_city)]['demand'] *
        delivery_cost[(warehouse, to_city)]['truck_unit_rate']
        for to_city in to_cities
        if (warehouse, to_city) in delivery_cost and y[(to_city, warehouse)].value() == 1
    )

    total_cost = inbound + adjusted_warehouse_cost + delivery
    total_inbound_cost += inbound
    total_warehouse_cost += adjusted_warehouse_cost
    total_delivery_cost += delivery

    results.append({
        "후보 창고 도시명": warehouse,
        "수요 박스 개수": sum(delivery_cost[(warehouse, to_city)]['demand'] for to_city in to_cities if (warehouse, to_city) in delivery_cost and y[(to_city, warehouse)].value() == 1),
        "운송비": delivery,
        "수입 물류비": inbound,
        "재고 증가 반영 창고비": adjusted_warehouse_cost,
        "총 물류비": total_cost
    })

# 총 비용 계산
total_costs = {
    "후보 창고 도시명": "총합계",
    "수요 박스 개수": sum(item["수요 박스 개수"] for item in results),
    "운송비": total_delivery_cost,
    "수입 물류비": total_inbound_cost,
    "재고 증가 반영 창고비": total_warehouse_cost,
    "총 물류비": total_inbound_cost + total_warehouse_cost + total_delivery_cost
}
results.append(total_costs)

# 결과를 데이터프레임으로 출력
df_after = pd.DataFrame(results)
pd.options.display.float_format = '{:,.2f}'.format
df_after

,후보 창고 도시명,수요 박스 개수,운송비,수입 물류비,재고 증가 반영 창고비,총 물류비
0,"AUSTIN, TX",2177041,"2,282,487.11","433,738.07","52,751.47","2,768,976.64"
1,"LOS ANGELES, CA",2751355,"2,889,411.13","955,165.00","324,309.38","4,168,885.51"
2,"PITTSBURGH, PA",3871611,"3,974,008.34","165,375.00","16,920.89","4,156,304.23"
3,총합계,8800007,"9,145,906.58","1,554,278.07","393,981.73","11,094,166.37"


# 시각화

In [ ]:
import plotly.graph_objects as go

# 데이터 준비
data = {
    "창고 도시명": ["AUSTIN, TX", "LOS ANGELES, CA", "PITTSBURGH, PA"],
    "재고 증가 반영 전 창고비": [2823.0, 17365.56, 905.04],
    "재고 증가 반영 후 창고비": [4889.58, 30078.03, 1567.57]
}

# 데이터프레임 생성
df = pd.DataFrame(data)

# 증감율 계산
df["증감율 (%)"] = ((df["재고 증가 반영 후 창고비"] - df["재고 증가 반영 전 창고비"]) / df["재고 증가 반영 전 창고비"]) * 100

# Plotly 그래프 생성
fig = go.Figure()

# 재고 증가 반영 전 창고비 막대
fig.add_trace(go.Bar(
    y=df["창고 도시명"],
    x=df["재고 증가 반영 전 창고비"],
    orientation='h',
    name="재고 증가 반영 전 창고비",
    marker=dict(color='lightblue')
))

# 재고 증가 반영 후 창고비 막대
fig.add_trace(go.Bar(
    y=df["창고 도시명"],
    x=df["재고 증가 반영 후 창고비"],
    orientation='h',
    name="재고 증가 반영 후 창고비",
    marker=dict(color='orange')
))

# 증감율 막대 추가
fig.add_trace(go.Bar(
    y=df["창고 도시명"],
    x=df["증감율 (%)"],
    orientation='h',
    name="증감율 (%)",
    marker=dict(color='green'),
    text=[f"{val:.1f}%" for val in df["증감율 (%)"]],
    textposition="outside"
))

# 레이아웃 설정
fig.update_layout(
    title="재고 증가 반영 전후 창고비 비교 및 증감율",
    xaxis_title="창고비",
    barmode='group',  # 그룹 모드로 설정하여 비교하기 쉽게 나란히 표시
    yaxis_title="창고 도시명",
)

# 그래프 표시
fig.show()



# 하버사인

distance를 haversine distance로 사용하고, 창고 수 3개일때의 재고 증가를 반영한 산출 코드입니다.

In [ ]:
import pandas as pd
import numpy as np

# 하버사인 거리 함수 정의 (단위: km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반지름 (단위: km)
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

# 데이터 불러오기
fromto_data = path['FROMTO(Q2)']
distance_data = path['Distance']

# 도시 이름 전처리 (공백 제거 및 소문자로 변환하여 일관성 확보)
fromto_data['From City'] = fromto_data['From City&State'].str.strip().str.lower()
fromto_data['To City'] = fromto_data['To City&State'].str.strip().str.lower()
distance_data['From City'] = distance_data['From City'].str.strip().str.lower()
distance_data['To City'] = distance_data['To City'].str.strip().str.lower()

# 필요한 창고 후보지와 수요 도시에 해당하는 행만 추출
warehouse_coords = distance_data[['From City', 'FROM CITY(위도)', 'FROM CITY(경도)']].drop_duplicates()
city_coords = distance_data[['To City', 'To CITY(위도)', 'To CITY(경도)']].drop_duplicates()

# 창고 후보지의 위도와 경도를 fromto_data에 병합
fromto_data = fromto_data.merge(
    warehouse_coords,
    left_on='From City',
    right_on='From City',
    how='left'
)

# 수요 도시의 위도와 경도를 fromto_data에 병합
fromto_data = fromto_data.merge(
    city_coords,
    left_on='To City',
    right_on='To City',
    how='left'
)

# 하버사인 거리를 계산하여 새로운 열에 추가
fromto_data['Haversine Distance'] = fromto_data.apply(
    lambda row: haversine(
        row['FROM CITY(위도)'], row['FROM CITY(경도)'],
        row['To CITY(위도)'], row['To CITY(경도)']
    ),
    axis=1
)

# 결과 확인 (38개 창고 후보 도시 * 297개 수요 도시)
print(fromto_data[['From City&State', 'To City&State', 'Haversine Distance']])



      From City&State      To City&State  Haversine Distance
0        New York, NY       New York, NY                0.00
1        New York, NY    Los Angeles, CA            3,957.33
2        New York, NY        Chicago, IL            1,155.06
3        New York, NY        Houston, TX            2,284.06
4        New York, NY   Philadelphia, PA              124.73
...               ...                ...                 ...
11281  Evansville, IN  Sandy Springs, GA              575.46
11282  Evansville, IN          Tyler, TX              943.37
11283  Evansville, IN     Las Cruces, NM            1,854.21
11284  Evansville, IN     South Bend, IN              424.20
11285  Evansville, IN     Woodbridge, NJ            1,173.96

[11286 rows x 3 columns]


In [ ]:
fromto_data

,City,From City&State,To City&State,TempD(직각거리),Inbound Unit Rate,TruckUnitRate,WHUnitRate,Demand,평균재고량,월평균재고량,...,Warehouse,Delivery,TOTAL,From City,To City,FROM CITY(위도),FROM CITY(경도),To CITY(위도),To CITY(경도),Haversine Distance
0,New York,"New York, NY","New York, NY",16.00,4.77,0.00,0.57,825722,"68,810.17","5,734.18",...,"471,723.26","28,348.75","4,441,017.92","new york, ny","new york, ny",40.66,-73.94,40.66,-73.94,0.00
1,New York,"New York, NY","Los Angeles, CA","5,487.54",4.77,0.00,0.57,382066,"31,838.83","2,653.24",...,"218,268.88","4,498,795.95","6,540,561.65","new york, ny","los angeles, ca",40.66,-73.94,34.02,-118.41,"3,957.33"
2,New York,"New York, NY","Chicago, IL","1,617.87",4.77,0.00,0.57,264741,"22,061.75","1,838.48",...,"151,242.78","919,061.96","2,333,841.33","new york, ny","chicago, il",40.66,-73.94,41.84,-87.68,"1,155.06"
3,New York,"New York, NY","Houston, TX","3,326.56",4.77,0.00,0.57,217788,"18,149.00","1,512.42",...,"124,419.19","1,554,566.51","2,718,428.43","new york, ny","houston, tx",40.66,-73.94,29.78,-95.39,"2,284.06"
4,New York,"New York, NY","Philadelphia, PA",189.62,4.77,0.00,0.57,151732,"12,644.33","1,053.69",...,"86,682.34","61,735.48","872,593.28","new york, ny","philadelphia, pa",40.66,-73.94,40.01,-75.13,124.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11281,Evansville,"Evansville, IN","Sandy Springs, GA",756.36,5.68,0.00,0.35,9910,825.83,68.82,...,"3,424.84","9,729.63","69,461.29","evansville, in","sandy springs, ga",37.99,-87.53,33.56,-84.23,575.46
11282,Evansville,"Evansville, IN","Tyler, TX","1,354.70",5.68,0.00,0.35,9863,821.92,68.49,...,"3,408.60","17,343.87","76,792.24","evansville, in","tyler, tx",37.99,-87.53,32.35,-95.30,943.37
11283,Evansville,"Evansville, IN","Las Cruces, NM","2,620.40",5.68,0.00,0.35,9862,821.83,68.49,...,"3,408.25","33,544.90","92,987.25","evansville, in","las cruces, nm",37.99,-87.53,32.32,-106.77,"1,854.21"
11284,Evansville,"Evansville, IN","South Bend, IN",466.15,5.68,0.00,0.35,9840,820.00,68.33,...,"3,400.65","5,954.02","65,263.76","evansville, in","south bend, in",37.99,-87.53,41.68,-86.27,424.20


In [ ]:
# 엑셀 파일에서 데이터 로드 (가정: 이미 로드된 warehouse_data 및 distance_data)
warehouse_data = path['WHCandidatdeCity']  # 창고 후보지 데이터
distance_data = fromto_data  # 거리 데이터
city_data = path['City']  # 수요 도시 데이터

# inbound_cost와 warehouse_cost를 딕셔너리로 변환
inbound_cost = {row['City&State'].strip().upper(): row['Inbound'] for _, row in warehouse_data.iterrows()}
warehouse_cost = {row['City&State'].strip().upper(): row['Warehouse'] for _, row in warehouse_data.iterrows()}

# 거리, 수요량, TruckUnitRate를 포함한 delivery_cost 딕셔너리 생성
delivery_cost = {
    (row['From City&State'].strip().upper(), row['To City&State'].strip().upper()): {
        'distance': row['Haversine Distance'],
        'demand': row['Demand'],
        'truck_unit_rate': row['TruckUnitRate']
    }
    for _, row in distance_data.iterrows()
}

# 최적화 문제 설정
model = LpProblem("Warehouse_Location_Optimization", LpMinimize)

# 결정 변수: 각 창고를 선택할지 여부 (0 또는 1)
warehouses = warehouse_data['City&State'].str.strip().str.upper().tolist()
x = LpVariable.dicts("SelectWarehouse", warehouses, cat='Binary')  # 창고 선택 변수

# 결정 변수: 각 도착 지역이 어떤 창고에서 물건을 받을지를 나타내는 변수 (0 또는 1)
to_cities = city_data['City&State'].str.strip().str.upper().tolist()
y = {
    (to_city, warehouse): LpVariable(f"Assign_{to_city}_to_{warehouse}", cat='Binary')
    for to_city in to_cities for warehouse in warehouses
}

# 목표 함수 설정: 총 물류비 최소화
model += lpSum([
    y[(to_city, warehouse)] * (
        delivery_cost[(warehouse, to_city)]['distance'] *
        delivery_cost[(warehouse, to_city)]['demand'] *
        delivery_cost[(warehouse, to_city)]['truck_unit_rate']
    )
    for to_city in to_cities
    for warehouse in warehouses
    if (warehouse, to_city) in delivery_cost
]) + lpSum([x[warehouse] * (inbound_cost[warehouse] + warehouse_cost[warehouse]) for warehouse in warehouses])

# 제약 조건 1: 각 도착 지역은 하나의 창고에서만 물건을 받을 수 있음
for to_city in to_cities:
    model += lpSum([y[(to_city, warehouse)] for warehouse in warehouses]) == 1

# 제약 조건 2: 창고가 선택되지 않으면 해당 창고로 할당할 수 없음
for warehouse in warehouses:
    for to_city in to_cities:
        model += y[(to_city, warehouse)] <= x[warehouse]

# 제약 조건 3: 정확히 3개의 창고를 선택
model += lpSum([x[warehouse] for warehouse in warehouses]) == 3

# 최적화 문제 해결
model.solve()

# 선택된 창고와 비용 계산
selected_warehouses = [warehouse for warehouse in warehouses if x[warehouse].value() == 1]

# 기본 재고량 설정 (예: 단일 창고일 때의 총 수요량 기반)
single_warehouse_inventory = sum(
    delivery_cost[(warehouse, to_city)]['demand']
    for to_city in to_cities
    for warehouse in selected_warehouses
    if (warehouse, to_city) in delivery_cost
)

# 창고 수 증가에 따른 재고 증가율 계산
inventory_increase_factor = (3 ** 0.5)  # 창고 수 3개일 때 제곱근 법칙 적용

# 결과 계산 및 데이터프레임 생성
results = []
total_inbound_cost = 0
total_warehouse_cost = 0
total_delivery_cost = 0

for warehouse in selected_warehouses:
    # 각 창고에 대한 비용 계산
    inbound = inbound_cost[warehouse]

    # 창고 수 증가에 따른 재고 증가 감안 창고비용 계산
    base_warehouse_cost = warehouse_cost[warehouse]
    adjusted_warehouse_cost = base_warehouse_cost * inventory_increase_factor

    delivery = sum(
        delivery_cost[(warehouse, to_city)]['distance'] *
        delivery_cost[(warehouse, to_city)]['demand'] *
        delivery_cost[(warehouse, to_city)]['truck_unit_rate']
        for to_city in to_cities
        if (warehouse, to_city) in delivery_cost and y[(to_city, warehouse)].value() == 1
    )

    total_cost = inbound + adjusted_warehouse_cost + delivery
    total_inbound_cost += inbound
    total_warehouse_cost += adjusted_warehouse_cost
    total_delivery_cost += delivery

    results.append({
        "후보 창고 도시명": warehouse,
        "수요 박스 개수": sum(delivery_cost[(warehouse, to_city)]['demand'] for to_city in to_cities if (warehouse, to_city) in delivery_cost and y[(to_city, warehouse)].value() == 1),
        "운송비": delivery,
        "수입 물류비": inbound,
        "재고 증가 반영 창고비": adjusted_warehouse_cost,
        "총 물류비": total_cost
    })

# 총 비용 계산
total_costs = {
    "후보 창고 도시명": "총합계",
    "수요 박스 개수": sum(item["수요 박스 개수"] for item in results),
    "운송비": total_delivery_cost,
    "수입 물류비": total_inbound_cost,
    "재고 증가 반영 창고비": total_warehouse_cost,
    "총 물류비": total_inbound_cost + total_warehouse_cost + total_delivery_cost
}
results.append(total_costs)

# 결과를 데이터프레임으로 출력
df_after2 = pd.DataFrame(results)
pd.options.display.float_format = '{:,.2f}'.format
df_after2

,후보 창고 도시명,수요 박스 개수,운송비,수입 물류비,재고 증가 반영 창고비,총 물류비
0,"NORFOLK, VA",2526046,"1,717,384.25","141,032.27","13,371.10","1,871,787.62"
1,"SALT LAKE CITY, UT",3163922,"4,027,916.32","90,705.57","11,630.29","4,130,252.18"
2,"ST. LOUIS, MO",3110039,"2,786,658.32","147,324.55","17,369.31","2,951,352.17"
3,총합계,8800007,"8,531,958.89","379,062.39","42,370.70","8,953,391.98"
